<table>
    <tr>
        <td><img src="./img/Macc.png" width="auto"/></td>
        <td>
            <table><tr>
            <h1 style="color:blue;text-align:center">Lógica para Ciencias de la Computación</h1></td>
            </tr></table>   
        <td>&nbsp;</td>
        <td>
            <table><tr>
            <tp><p style="font-size:150%;text-align:center">Taller</p></tp>
            <tp><p style="font-size:150%;text-align:center">Clasificación de fórmulas</p></tp>
            </tr></table>
        </td>
    </tr>
</table>

---

# Objetivos <a class="anchor" id="inicio"></a>

En este notebook vamos a utilizar las herramientas de evaluación del valor de verdad de una fórmula que hemos desarrollado hasta ahora, para clasificarla dentro de una de las categorías que aprendimos en clase. A saber, fórmulas válidas, satisfacibles, falseables, contingentes e inválidas.

# Secciones

1. [SATproblem.](#sat)
2. [Clasificación de fórmulas.](#clas)
3. [Implicación lógica.](#imp)
4. [Eficiencia de los algoritmos (opcional).](#efi)

# SATproblem <a class="anchor" id="sat"></a>

([Volver al inicio](#inicio))

Hemos visto que una fórmula $A$ es satisacible sii existe una interpretación $I$ tal que $A$.valor$(I)=\texttt{True}$. En lógica computacional, determinar si una fórmula $A$ es satisfacible se conoce como el SATproblem. En teoría de la complejidad, este problema es importante, toda vez que fue el primer problema que se demostró que es NP-completo. Pero esto es parte de otro curso. Para nosotros el SATproblem es importante porque su solución será la base tanto para la clasificación de fórmulas como para la resolución de problemas. 

Los algoritmos que, dada una fórmula $A$, búscan una interpretación $I$ tal que $A$.valor$(I)=\texttt{True}$, se conocen como SATsolvers. En este notebook construiremos el primer SATsolver con base en las herramientas que tenemos a nuestra disposición.

**Ejercicio 1:** 

Implemente un método `SATtabla` el cual, dada una fórmula $A$, devuelve una interpretación $I$ tal que $A$.valor$(I)=\texttt{True}$, si es que $A$ es satisfacible. Si no, `SATtabla` debe devolver `None`. Realice este método recorriendo las interpretaciones posibles de la fórmula una a una hasta encontrar una que la haga verdadera.

Verifique su implementación para corroborar que las siguientes fórmulas son satisfacibles:

**Nota importante:** Estas fórmulas usan simplificación de paréntesis. No olvide que para usar `inorder_to_tree` todos los paréntesis deben estar incluidos.


* $\neg p\wedge q$
* $\bigl(p\vee(q\wedge r)\bigr)\wedge\bigl((\neg p\vee \neg q)\wedge(\neg p\vee\neg r)\bigr)$
* $(\neg p\leftrightarrow (q\vee r))\wedge(\neg q\vee\neg p)\wedge (\neg r\vee p)\wedge (q\vee r\vee\neg p)$

Y que las siguientes fórmulas no son satisfacibles:

* $p\wedge (p\to\neg p)$
* $\neg(\neg(p\wedge q)\to (\neg p\vee\neg q))$
* $(p\leftrightarrow (q\to r))\wedge\neg((q\vee p)\wedge (\neg r\vee p)\wedge (\neg q\vee r\vee\neg p))$



In [3]:
from itertools import product
from Logica import *

In [4]:
def SATtabla(A: Formula):
    letras = A.letras()
    num_letras = len(letras)
    interpretaciones = [dict(zip(letras, c )) for c in product([True, False], repeat=num_letras)]
    for I in interpretaciones:
        if A.valor(I):
            return I
    return None

setattr(Formula, 'SATtabla', SATtabla)

In [5]:
A = inorder_to_tree('(-pYq)')
print(A.SATtabla())

A = inorder_to_tree('((pO(qYr))Y((-pO-q)Y(-pO-r)))')
print(A.SATtabla())

A = inorder_to_tree('(((-p=(qOr))Y(-qO-p))Y((-rOp)Y((qOr)O-p))')
print(A.SATtabla())

A = inorder_to_tree('(pY(p>-p))')
print(A.SATtabla())

A = inorder_to_tree('-(-(pYq)>(-pO-q))')
print(A.SATtabla())

A = inorder_to_tree('(((p=(q>r))Y-(qOp))Y-((-rOp)Y((-qOr)O-p))')
print(A.SATtabla())


{'q': True, 'p': False}
{'q': True, 'p': False, 'r': True}
{'-': True, 'p': True, 'q': False, 'r': False}
None
None
None


---

# Clasificación de fórmulas <a class="anchor" id="clas"></a>

([Volver al inicio](#inicio))

En clase vimos la siguiente clasificación de una fórmula $A$, con base en la cantidad de interpretaciones que la hacen verdadera:

<img src="./img/clasificacion.png" width="auto"/>



**Ejercicio 2:**

Complete el siguiente diagrama de flujo que permite realizar la clasificación "$A$ es válida", "$A$ es insatisfacible", "$A$ es contingente", el cual está basado en el uso de un SATsolver para decidir si una fórmula es satisfacible o no:

<img src="./img/flow1.png" width="auto"/>

-A NO es satisfacible -> A es valida

-A es satisfacible -> A es contingente

---

**Ejercicio 3:**

Utilice `if`s anidados para implementar el anterior diagrama de flujo en un método `clasifica` de la clase `Formula` que devuelve un string "Válida", "Insatisfacible" o "Contingente" de acuerdo a como corresponda.

Compruebe su implementación con las siguientes fórmulas:

* $p\wedge (p\to\neg p)$ es insatisfacible.
* $p\to (\neg p\to q)$ es válida.
* $p\wedge (q\to \neg p)$ es contingente.

In [6]:
def clasifica(self: Formula):
    if self.SATtabla() == None:
        return 'INSATISFACIBLE'
    else:
        if Negacion(self).SATtabla() == None:
            return 'VALIDA'
        else:
            return 'CONTINGENTE'

setattr(Formula, 'clasifica', clasifica)

In [7]:
A = inorder_to_tree('(pY(p>-p))')
print(A.clasifica())

A = inorder_to_tree('(p>(-p>q))')
print(A.clasifica())

A = inorder_to_tree('(pY(q>-p))')
print(A.clasifica())


INSATISFACIBLE
VALIDA
CONTINGENTE


---

# Implicación lógica <a class="anchor" id="imp"></a>

([Volver al inicio](#inicio))

Recordemos que en clase vimos la siguiente definición de implicación lógica:

$U\models B$ sii para toda interpretación $I$, si $A_i.{valor}(I)=\texttt{True}$ para todo $A_i\in U$,        entonces $B.{valor}(I)=\texttt{True}$.

También mencionamos que para verificar si hay o no implicación, es muy útil usar el siguiente teorema:

*Proposición* Sea $B$ una fórmula y $U=\{A_1,\ldots,A_n\}$ un conjunto de fórmulas:

$U\models B$ sii $\bigl(A_1\wedge\ldots\wedge A_n\bigr)\to B$ es válida.

**Ejercicio 4:**

Complete el siguiente diagrama de flujo para determinar si hay implicación lógica entre un conjunto de fórmulas $U$ y una fórmula $B$ con base en un SATsolver.

<img src="./img/flow_imp.png" width="auto"/>

if -(Ytoria(U) -> B) es insatisfacible:

    U no implica a B

else

    U implica a B

---

**Ejercicio 5:**

Implemente una función `imp_log` que recibe como argumentos una lista de fórmulas `[A1,...,An]` y una fórmula `B` y devuelve verdadero sii $\{A_1,\ldots,A_n\}\models B$.

Compruebe su código con las siguientes implicaciones:

* $U\,{=}\,\{p, \neg q\}$; $B\,{=}\,\neg(p\to q)$.
* $U\,{=}\,\{p\to q, \neg r, q\to r\}$; $B\,{=}\,\neg p$.
* $U\,{=}\,\{r\vee s, \neg s\wedge\neg r, p\vee q, p\to q, r\to s\}$; $B\,{=}\,\neg p\wedge\neg q$.

Y compruebe que las siguientes no son implicaciones:

* $U\,{=}\,\{p\wedge r,\ (p\wedge q)\rightarrow\lnot r \}$; $B\,{=}\, q$
* $U\,{=}\,\{p\rightarrow q,\ p\wedge(q\rightarrow r),\ r\rightarrow s \}$; $B\,{=}\,p\wedge\lnot s$

In [8]:
def imp_log(U: list[str], B: str):
    antecedente = inorder_to_tree(Ytoria(U))
    consecuencia = inorder_to_tree(B)
    proposicion = Negacion(Binario('>', antecedente, consecuencia))
    if proposicion.SATtabla() == None:
        return True
    else:
        return False

In [15]:
U = {'p', '-q'}
B = '-(p>q)'
print(imp_log(U,B))

U = {'(p>q)', '-r', '(q>r)'}
B = '-p'
print(imp_log(U,B))

U = {'(rOs)', '(-sY-r)', '(pOq)', '(p>q)', '(r>s)'}
B = '(-pY-q)'
print(imp_log(U,B))

U = {'(pYr)', '((pYq)>-r)'}
B = 'q'
print(imp_log(U,B))

U = {'(p>q)', '(pY(q>r))', '(r>s)'}
B = '-(p>q)'
print(imp_log(U,B))


True
True
True
False
False


---

# Sección opcional: Eficiencia de los algoritmos <a class="anchor" id="efi"></a>


([Volver al inicio](#inicio))

Hemos afirmado anteriormente que la eficiencia de los algoritmos es un tema central en las ciencias de la computación. Vamos a comprobar que nuestro SATsolver hasta el momento tiene un comportamiento exponencial de tiempo de ejecución con base en el número de letras proposicionales distintas que haya en la fórmula de entrada. Así pues, vamos a ver que el tiempo que demora el método `SATtabla` al correr sobre una fórmula `A` depende de manera exponencial en el tamaño de `A.letras()`.

Definimos nuestras funciones para medir tiempos en una lista de entradas: 

In [10]:
from time import time
import seaborn as sns
import pandas as pd

def obtiene_tiempos(fun, args, num_it=100):
    tiempos_fun = []
    for i in range(num_it):
        arranca = time()
        x = fun(*args)
        para = time()
        tiempos_fun.append(para - arranca)
    return tiempos_fun

def compara_entradas(fun, lista_args, N=100):
    entradas = []
    tiempos = []
    for i, args in enumerate(lista_args):
        t = obtiene_tiempos(fun, [args], N)
        tiempos += t
        n = len(t)
        entradas += [i+1]*n
    df = pd.DataFrame({'Long_entrada':entradas, 'Tiempo_prom':tiempos})
    print(df.head())
    sns.lineplot(x='Long_entrada',y='Tiempo_prom',data=df)
    plt.show()

ModuleNotFoundError: No module named 'seaborn'

Creamos nuestra lista de fórmulas:

In [ ]:
from random import uniform, choice

cantidad = 20
letras = [chr(i) for i in range(256, 256+cantidad)]
lista = [letras[0]]
formula = letras[0]
for p in letras[1:]:
    neg = '-' if uniform(0,1) > .5 else ''
    conectivo = choice(['Y','O','>','='])
    formula = "(" + formula + conectivo + neg + p + ")"
    lista.append(formula)

lista

Y, finalmente, medimos los tiempos de ejecución para cada fórmula:

In [ ]:
fun = lambda formula: inorder_to_tree(formula).SATtabla()
compara_entradas(fun, lista)

En esta gráfica es muy fácil apreciar que, a medida que aumentamos el número de letras proposicionales distintas en la fórmula de entrada, el algoritmo se tarda cada vez más en determinar si ella es satisfacible. Más aún, la relación entre el número de letras y el tiempo es claramente exponencial.

Como conclusión, vemos que si necesitamos una gran cantidad de letras proposicionales, nuestro SATsolver `SATtabla` resultará completamente ineficiente.

---